# Encrypting AES BIE1

### Bitcoin conventions

In [2]:
import binascii
import hashlib
import ecdsa
import codecs

# secp256k1, http://www.oid-info.com/get/1.3.132.0.10
p_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
r_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
b_ec = 0x0000000000000000000000000000000000000000000000000000000000000007
a_ec = 0x0000000000000000000000000000000000000000000000000000000000000000
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
curve_secp256k1 = ecdsa.ellipticcurve.CurveFp(p_ec, a_ec, b_ec)
generator_secp256k1 = ecdsa.ellipticcurve.Point(curve_secp256k1, Gx, Gy, r_ec)
oid_secp256k1 = (1, 3, 132, 0, 10)
SECP256k1 = ecdsa.curves.Curve("SECP256k1", curve_secp256k1, generator_secp256k1, oid_secp256k1)

### Establishing a shared secret

In [3]:
SA = 68391376603652558019677085321245066310756998499824362348774548882158845001985
PA = SA * generator_secp256k1

SB = 95741376603652558019677085321245066310756998499824362348774548882158845001985
PB = SB * generator_secp256k1

In [4]:
# Shared secret

SAB = SA * PB

In [5]:
# Check shared secrets are equal if calculated by either Alice or Bob

(SA * PB) == (SB * PA)

True

In [6]:
def get_point_pubkey(point_input):
    if (point_input.y() % 2) == 1:
        result = '03' + '%064x' % point_input.x()
    else:
        result = '02' + '%064x' % point_input.x()
    return binascii.hexlify(codecs.decode(result, 'hex'))

In [7]:
str(SAB.x())

'24547207253151602529617225665887624910292130993218504679930071992194063220891'

In [8]:
S = get_point_pubkey(SAB)
S

b'0236453a194181e14a3333f9e284afefffc05eec8680e25c6bf5432aa53d65409b'

### Symmetric keys

In [9]:
# Base 64
print(hashlib.sha512(S).digest())

# Hex
print(binascii.hexlify(hashlib.sha512(S).digest()))

b'V\xe4\xce~\x93\xa9\x92\x14\xeb\x81\xf8\xba\xde\x11\xa0\xdc\xe9T\xb1R\r\xb5\xd8\xb1\'\xba\x91^?f\x95\xe4*o\xcfv&\xebX"\xf5yf\xe6\x8a\xffe\x9257\xbf\xa1N\xd7R!\xae\x97b\xc6\xd4*6\n'
b'56e4ce7e93a99214eb81f8bade11a0dce954b1520db5d8b127ba915e3f6695e42a6fcf7626eb5822f57966e68aff65923537bfa14ed75221ae9762c6d42a360a'


In [10]:
hashlib.sha512(S).digest()

b'V\xe4\xce~\x93\xa9\x92\x14\xeb\x81\xf8\xba\xde\x11\xa0\xdc\xe9T\xb1R\r\xb5\xd8\xb1\'\xba\x91^?f\x95\xe4*o\xcfv&\xebX"\xf5yf\xe6\x8a\xffe\x9257\xbf\xa1N\xd7R!\xae\x97b\xc6\xd4*6\n'

In [11]:
binascii.hexlify(hashlib.sha512(S).digest())

b'56e4ce7e93a99214eb81f8bade11a0dce954b1520db5d8b127ba915e3f6695e42a6fcf7626eb5822f57966e68aff65923537bfa14ed75221ae9762c6d42a360a'

In [116]:
IV = hashlib.sha512(S).digest()[0:16]
KAES = hashlib.sha512(S).digest()[16:32]
KHMAC = hashlib.sha512(S).digest()[32:48]

### Encrypt message

In [14]:
from Crypto.Cipher import AES

In [58]:
# Encoding and decoding text

print(codecs.ascii_encode('The answer is no')[0])
print(binascii.hexlify(codecs.ascii_encode('The answer is no')[0]))
print(binascii.unhexlify(binascii.hexlify(codecs.ascii_encode('The answer is no')[0])))

b'The answer is no'
b'54686520616e73776572206973206e6f'
b'The answer is no'


In [118]:
# Encryption working with KAES and IV

message = KAES
print(message)
aes = AES.new(KAES, AES.MODE_CBC, IV)
ciphertext = IV + aes.encrypt(message)
print(ciphertext)
print(aes.decrypt(ciphertext)[16:])

b"\xe9T\xb1R\r\xb5\xd8\xb1'\xba\x91^?f\x95\xe4"
b'V\xe4\xce~\x93\xa9\x92\x14\xeb\x81\xf8\xba\xde\x11\xa0\xdc\xc1=\xefW\xfe\xc3\xf6\x8bl\xd1\x88\xccnW\xdf;'
b"\xe9T\xb1R\r\xb5\xd8\xb1'\xba\x91^?f\x95\xe4"


In [129]:
print(binascii.b2a_qp(IV))
print(binascii.a2b_qp(binascii.b2a_qp(IV)))

print(binascii.a2b_qp(codecs.ascii_encode('The answer is no')[0]))

b'V=E4=CE~=93=A9=92=14=EB=81=F8=BA=DE=11=A0=DC'
b'V\xe4\xce~\x93\xa9\x92\x14\xeb\x81\xf8\xba\xde\x11\xa0\xdc'
b'The answer is no'


In [118]:
# Encryption working with KAES and IV

message = KAES
print(message)
aes = AES.new(KAES, AES.MODE_CBC, IV)
ciphertext = IV + aes.encrypt(message)
print(ciphertext)
print(aes.decrypt(ciphertext)[16:])

b"\xe9T\xb1R\r\xb5\xd8\xb1'\xba\x91^?f\x95\xe4"
b'V\xe4\xce~\x93\xa9\x92\x14\xeb\x81\xf8\xba\xde\x11\xa0\xdc\xc1=\xefW\xfe\xc3\xf6\x8bl\xd1\x88\xccnW\xdf;'
b"\xe9T\xb1R\r\xb5\xd8\xb1'\xba\x91^?f\x95\xe4"


In [135]:
from Crypto import Random
key = b'Sixteen byte key'
iv = Random.new().read(AES.block_size)

message = b'Sixteen byte msg'
print(message)
aes = AES.new(KAES, AES.MODE_CBC, IV)
ciphertext = IV + aes.encrypt(message)
print(ciphertext)
print(aes.decrypt(ciphertext)[16:])

b'Sixteen byte msg'
b'V\xe4\xce~\x93\xa9\x92\x14\xeb\x81\xf8\xba\xde\x11\xa0\xdc\x9b\xe1:aE\x1e\xa2\xf3\xed\xa7YH\x08\xab,\x1b'
b'Sixteen byte msg'


In [131]:
iv

b'\xf5D9\xef\x16\xe5 \x1bb|\xd29p\xcb\x7f\x1a'

## Working solution

In [151]:
aes = AES.new(KAES, AES.MODE_CBC, IV)
message = b'The answer is no'    # I think this is ascii encoded bits
print(message)
print(binascii.hexlify(message))
ciphertext = aes.encrypt(message)
print(binascii.hexlify(ciphertext))
print(binascii.hexlify(aes.decrypt(IV + ciphertext)[16:]))
print(aes.decrypt(IV + ciphertext)[16:])

b'The answer is no'
b'54686520616e73776572206973206e6f'
b'acf4eb9dba1920fcd598bb9669f20ae9'
b'54686520616e73776572206973206e6f'
b'The answer is no'


In [149]:
# Check that we could have encrypted the message directly

aes = AES.new(KAES, AES.MODE_CBC, IV)
message = b'54686520616e73776572206973206e6f'    # I think this is ascii encoded bits
print(message)
ciphertext = aes.encrypt(message)
print(binascii.hexlify(ciphertext))
print(aes.decrypt(IV + ciphertext)[16:])

b'54686520616e73776572206973206e6f'
b'6d622ce583dee1c4618a026ced39dea07fd95e756af53a12f41869eda78e6dba'
b'54686520616e73776572206973206e6f'
